In [13]:
from langchain.memory import ChatMessageHistory

history = ChatMessageHistory()

history.add_user_message("hi!")

history.add_ai_message("whats up?")

In [14]:
history

ChatMessageHistory(messages=[HumanMessage(content='hi!'), AIMessage(content='whats up?')])

# Memory Types

- source: https://python.langchain.com/docs/modules/memory/types/buffer

In [15]:
from langchain.memory import ConversationBufferMemory

In [19]:
memory = ConversationBufferMemory()
memory.save_context({"input": "hi"}, {"output:": "whats up?"})

In [22]:
memory.load_memory_variables({})

{'history': 'Human: hi\nAI: whats up?'}

Getting the history as a list of messages (useful for chat models).

In [24]:
memory = ConversationBufferMemory(return_messages=True)
memory.save_context({"input": "hi"}, {"output": "whats up"})
memory.load_memory_variables({})

{'history': [HumanMessage(content='hi'), AIMessage(content='whats up')]}

Using memory in a chain:

In [28]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain


llm = ChatOpenAI(temperature=0)
conversation = ConversationChain(
    llm=llm,
    verbose=True,
    memory=ConversationBufferMemory()
)

In [29]:
conversation.predict(input="Hi there!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there!
AI:

> Finished chain.


'Hello! How can I assist you today?'

In [31]:
conversation.predict(input="I'm doing well! You should know my names is Lucas. I am a machine learning engineer who loves to write code to interact with LLMs.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI: Hello! How can I assist you today?
Human: I'm doing well! Just having a conversation with an AI.
AI: That's great to hear! I'm here to chat and provide information. What would you like to talk about?
Human: I'm doing well! You should know my names is Lucas. I am a machine learning engineer who loves to write code to interact with LLMs.
AI:

> Finished chain.


"Nice to meet you, Lucas! As a machine learning engineer, you must have a deep understanding of how LLMs work. LLMs, or Language Models, are a type of AI model that can generate human-like text based on the input it receives. They are trained on vast amounts of data and can be used for various tasks such as generating conversational responses, writing articles, or even composing music. It's fascinating how LLMs can understand and generate text that is coherent and contextually relevant. Is there anything specific you would like to know or discuss about LLMs?"

In [33]:
conversation.predict(input="what is my name and what are my interests?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI: Hello! How can I assist you today?
Human: I'm doing well! Just having a conversation with an AI.
AI: That's great to hear! I'm here to chat and provide information. What would you like to talk about?
Human: I'm doing well! You should know my names is Lucas. I am a machine learning engineer who loves to write code to interact with LLMs.
AI: Nice to meet you, Lucas! As a machine learning engineer, you must have a deep understanding of how LLMs work. LLMs, or Language Models, are a type of AI model that can generate human-like text based on the input it receives. They are trained on vast amounts of data and can be used for various tasks such as genera

"Your name is Lucas, as you mentioned earlier. As for your interests, you mentioned that you are a machine learning engineer who loves to write code to interact with LLMs. This suggests that you have a passion for artificial intelligence and natural language processing. It's great to see that you are actively involved in the field and enjoy working with LLMs. Is there anything else you would like to share or discuss?"

You can set a buffer window (over the last k interactions for example) to avoid having the buffer getting too large:

In [37]:
# source: https://python.langchain.com/docs/modules/memory/types/buffer_window
from langchain.memory import ConversationBufferWindowMemory

# the last parameter is specially for when working with chat models.
memory = ConversationBufferWindowMemory( k=1, return_messages=True)
memory.save_context({"input": "hi"}, {"output": "whats up"})
memory.save_context({"input": "not much you"}, {"output": "not much"})
memory.load_memory_variables({})

{'history': [HumanMessage(content='not much you'),
  AIMessage(content='not much')]}

Using that inside a chain:

In [39]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain


conversation_with_summary = ConversationChain(
    llm=ChatOpenAI(temperature=0),
    # We set a low k=2, to only keep the last 2 interactions in memory
    memory=ConversationBufferWindowMemory(k=2),
    verbose=True
)
conversation_with_summary.predict(input="Hi, what's up?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, what's up?
AI:

> Finished chain.


"Hello! I'm an AI designed to assist with various tasks and provide information. Right now, I'm here to chat with you. How can I help you today?"

# Memory in Agents

In [41]:
# source: https://python.langchain.com/docs/modules/memory/agent_with_memory
from langchain.agents import AgentExecutor, Tool, ZeroShotAgent
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory
from langchain.tools import DuckDuckGoSearchRun

tools = [DuckDuckGoSearchRun()]

prefix = """Have a conversation with a human, answering the following questions as best you can. You have access to the following tools:"""
suffix = """Begin!"

{chat_history}
Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input", "chat_history", "agent_scratchpad"],
)
memory = ConversationBufferMemory(memory_key="chat_history")

llm_chain = LLMChain(llm=OpenAI(temperature=0), prompt=prompt)
agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)
agent_chain = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True, memory=memory
)

agent_chain.run(input="How many people live in canada?")



> Entering new AgentExecutor chain...
Thought: I should look up the population of Canada
Action: duckduckgo_search
Action Input: population of Canada
Observation: The 2021 Canadian census counted a total population of 36,991,981, an increase of around 5.2 per cent over the 2016 figure. [7] [8] Between 1990 and 2008, the population increased by 5.6 million, equivalent to 20.4 per cent overall growth. [9] Historical population overview [ edit] Indigenous peoples [ edit] The 2021 Canadian census enumerated a total population of 36,991,981, an increase of around 5.2 percent over the 2016 figure, [4] Between 2011 and May 2016, Canada's population grew by 1.7 million people, with immigrants accounting for two-thirds of the increase. [5] Canada's population was estimated at 39,566,248 on January 1, 2023, after a record population growth of 1,050,110 people from January 1, 2022, to January 1, 2023. Canada's population was estimated at 39,292,355 people on October 1, 2022, an increase of 362,

"As of June 16, 2023, Canada's population was estimated to be 40 million people."

In [42]:

# testing the memory of the agent
agent_chain.run(input="what is their national anthem called?")



> Entering new AgentExecutor chain...
Thought: I should look up the national anthem of Canada
Action: duckduckgo_search
Action Input: "national anthem of Canada"
Observation: O Canada, national anthem of Canada. It was proclaimed the official national anthem on July 1, 1980. "God Save the Queen" remains the royal anthem of Canada. "O Canada" (French: Ô Canada) is the national anthem of Canada. The song was originally commissioned by Lieutenant Governor of Quebec Théodore Robitaille for the 1880 Saint-Jean-Baptiste Day... Canada's national anthem has both French and English versions, to honour the country's bilingual past and present. Here are the history and lyrics to 'O Canada'. Listen to this article Loading audio... On their visits to Canada in the early 1900s, English kings would salute 'O Canada' as if it were a national anthem. National Anthem of Canada - ''O Canada'' Duke of San Pablo 30 subscribers 4 64 views 2 weeks ago Captions: English translation of the French lyrics "O C

'The national anthem of Canada is called "O Canada".'

Cool examples for how to extend this idea of memory:

- [Conversation knowledge graphs](https://python.langchain.com/docs/modules/memory/types/kg)
- [Conversation summaries](https://python.langchain.com/docs/modules/memory/types/summary)